In [1]:
import pandas as pd
import numpy as np
import re

import plotly.express as px
import ipywidgets as widgets

pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)

In [2]:
meta = pd.read_csv("./input_folder/KS/meta_final2.tsv", sep = "\t", index_col = 0)
rename_to = {v: k for k, v in meta["old_name"].to_dict().items()}

In [3]:
path = "input_folder/KS/16S_predicted.tsv.gz"
df = pd.read_csv(path, compression="gzip", header=0, sep="\t", index_col = 0)
df.sort_values("16S_rRNA_Count", ascending = False, inplace = True)

In [4]:
taxo = pd.read_csv("input_folder/KS/taxonomy.tsv", sep = "\t", index_col = 0)

## Taxonomy with NSTI value above 2

In [5]:
a = pd.DataFrame(
    taxo.loc[
            df.loc[df["metadata_NSTI"] > 2, :].index.tolist(), :
        ].value_counts("Taxon")
)

a.columns = ["Number of ASVs"]
a = a.loc[sorted(a.index.tolist()), :]
a

,Number of ASVs
Taxon,
d__Archaea,3
d__Archaea; p__Aenigmarchaeota; c__Aenigmarchaeia; o__Aenigmarchaeales; f__Aenigmarchaeales; g__Aenigmarchaeales; s__uncultured_archaeon,4
d__Archaea; p__Aenigmarchaeota; c__Aenigmarchaeia; o__Aenigmarchaeales; f__Aenigmarchaeales; g__Candidatus_Aenigmarchaeum; s__uncultured_archaeon,2
d__Archaea; p__Crenarchaeota; c__Bathyarchaeia; o__Bathyarchaeia; f__Bathyarchaeia; g__Bathyarchaeia; s__uncultured_archaeon,1
d__Archaea; p__Nanoarchaeota; c__Nanoarchaeia; o__Woesearchaeales; f__GW2011_GWC1_47_15; g__GW2011_GWC1_47_15; s__uncultured_archaeon,1
d__Archaea; p__Nanoarchaeota; c__Nanoarchaeia; o__Woesearchaeales; f__SCGC_AAA011-D5; g__SCGC_AAA011-D5,4
d__Archaea; p__Nanoarchaeota; c__Nanoarchaeia; o__Woesearchaeales; f__SCGC_AAA011-D5; g__SCGC_AAA011-D5; s__Candidatus_Pacearchaeota,1
d__Archaea; p__Nanoarchaeota; c__Nanoarchaeia; o__Woesearchaeales; f__SCGC_AAA011-D5; g__SCGC_AAA011-D5; s__uncultured_archaeon,1
d__Archaea; p__Nanoarchaeota; c__Nanoarchaeia; o__Woesearchaeales; f__SCGC_AAA011-D5; g__SCGC_AAA011-D5; s__uncultured_euryarchaeote,4


## Data transformation+ordination on predicted functions (Kegg Ontology)
### KS samples only

In [7]:
bray_pca_df = pd.read_csv("input_folder/KS/functional_metagenome/KO/bray_pca.tsv", sep = "\t")
metadata_dropdown_bray_pca = widgets.Dropdown(options=list(meta.columns), description='Select Metadata:')

def update_color_bray_pca(selected_metadata):
    fig = px.scatter_3d(bray_pca_df, x='PC1', y='PC2', z = "PC3",
                     color = selected_metadata,
                     text='uniqID', title='Bray Curtis + PCA Analysis')
    fig.update_traces(textposition='top center')
    fig.update_layout(width = 800, height = 800)
    fig.show()

# Observe the changes and update the plot on metadata selection
widgets.interactive(update_color_bray_pca, selected_metadata=metadata_dropdown_bray_pca)

interactive(children=(Dropdown(description='Select Metadata:', options=('uniqID', 'filename_prefix', 'temperat…

In [ ]:
metadata_dropdown_bray_pca = widgets.Dropdown(options=list(meta.columns),
                                              description='Select Metadata:')

def update_color_bray_pca(selected_metadata):
    fig = px.scatter(bray_pca_df, x='PC1', y='PC2',
                     color = selected_metadata,
                     text='uniqID', title='Bray Curtis + PCA Analysis')
    fig.update_traces(textposition='top center')
    fig.update_layout(width = 800, height = 800)
    fig.show()

# Observe the changes and update the plot on metadata selection
widgets.interactive(update_color_bray_pca, selected_metadata=metadata_dropdown_bray_pca)